In [1]:
import os
import sys
parent_dir = os.path.abspath('../../..')
sys.path.append(parent_dir)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # or "true" depending on your requirement
import os
import sys
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from eval_utils import compare_MLE, data_profiling, cate_info, col_check,  transform_label
from sklearn.preprocessing import LabelBinarizer
import os
import json
import pickle
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer, TVAESynthesizer
from sdmetrics.reports.single_table import QualityReport
from be_great import GReaT

2025-04-09 04:25:42.253618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744172742.272744   20715 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744172742.278694   20715 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744172742.294292   20715 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744172742.294309   20715 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744172742.294311   20715 computation_placer.cc:177] computation placer alr

In [3]:
path = "../../sample/Adult/"
df25 = pd.read_csv(path + 'df25.csv')
df50 = pd.read_csv(path + 'df50.csv')
df25t = pd.read_csv(path + 'df25_test.csv')
df50t = pd.read_csv(path + 'df50_test.csv')

In [5]:
data_lst = [df25, df50]
test_lst = [df25t, df50t]
num_lst = [25, 50]
seed_lst = list(range(5))

x_cols = ['age', 'workclass', 'education', 'education-num',
          'marital-status', 'occupation', 'relationship', 'race', 'sex',
          'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
y_col = 'Income'
cate_var = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'label']
bool_var = []
cols = x_cols + [y_col]
num_var = list((set(cols) - set(cate_var)) - set(bool_var))

In [6]:
epoch_lst = [1000, 1000]
for i in range(2):
    num = num_lst[i]
    sampled_data = data_lst[i]
    sampled_data = sampled_data[cols].copy()
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(sampled_data[cols])
    metadata_dict = metadata.to_dict()
    ctgan_model = CTGANSynthesizer(metadata, cuda=True)
    ctgan_model.fit(sampled_data[cols])
    tvae_model = TVAESynthesizer(metadata, cuda=True)
    tvae_model.fit(sampled_data[cols])

    great_model = GReaT(llm='distilgpt2', batch_size = 40, epochs=epoch_lst[i])
    great_model.fit(sampled_data[cols])

    for j in range(len(seed_lst)):
        seed = seed_lst[j]
        # CTGAN
        #ctgan_model.set_random_state(seed)
        ctgan_syn = ctgan_model.sample(len(sampled_data))
        ctgan_syn.reset_index(inplace=True, drop=True)
        ctgan_syn.to_csv(f'gen/Adult/{num}/CTGAN/df_{seed}.csv', index=False)
        
        #tvae_model.set_random_state(seed)
        tvae_syn = tvae_model.sample(len(sampled_data))
        tvae_syn.reset_index(inplace=True, drop=True)
        tvae_syn.to_csv(f'gen/Adult/{num}/TVAE/df_{seed}.csv', index=False)

        # Be-great
        great_syn_df = great_model.sample(len(sampled_data), max_length=4000)
        great_syn_df.reset_index(inplace=True, drop=True)
        great_syn_df.to_csv(f'gen/Adult/{num}/BeGreaT/df_{seed}.csv', index=False)

/opt/conda/envs/py310/lib/python3.10/site-packages/sdv/single_table/base.py:92: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
/opt/conda/envs/py310/lib/python3.10/site-packages/torch/autograd/graph.py:768: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.508300
1000,0.335600


  0%|          | 0/25 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
86it [00:00, 153.07it/s]              
77it [00:00, 150.61it/s]              
87it [00:00, 170.93it/s]              
84it [00:00, 163.38it/s]              
78it [00:00, 148.77it/s]              
/opt/conda/envs/py310/lib/python3.10/site-packages/sdv/single_table/base.py:92: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.560500
1000,0.379800
1500,0.344600
2000,0.330800


94it [00:00, 172.93it/s]              
87it [00:00, 170.08it/s]              
86it [00:00, 164.95it/s]              
92it [00:00, 180.10it/s]              
89it [00:00, 162.24it/s]              
